In [1]:
%matplotlib inline

from cocoapi.PythonAPI.pycocotools.coco import COCO
#from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import os
import tensorflow as tf
from datasets import data as dataset
from models.nn import YOLO as ConvNet
from learning.optimizers import MomentumOptimizer as Optimizer
# from learning.optimizers import AdamOptimizer as Optimizer
from learning.evaluators import RecallEvaluator as Evaluator

dataDir='/home/intern/data/coco'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

coco=COCO(annFile)

#file name extraction
AA = coco.getImgIds();

print(len(AA))
print(type(AA))
print(AA)

BB = coco.getAnnIds(imgIds=[AA[3]])#bboxes for one file

print(len(BB))
print(type(BB))
print(BB)

CC = coco.loadAnns(BB)

print(len(CC))
print(type(CC))
print(CC)

In [2]:
root_dir = os.path.join('/home/intern/data/coco') # FIXME
trainval_dir = os.path.join(root_dir, 'images')

In [3]:
anchors = dataset.load_json(os.path.join('/home/intern/data/face/train', 'anchors.json'))

In [4]:
IM_SIZE = (416, 416)
NUM_CLASSES = 80

In [5]:
X_trainval, y_trainval = dataset.read_data(trainval_dir, IM_SIZE)

loading annotations into memory...
Done (t=0.76s)
creating index...
index created!


In [6]:
trainval_size = X_trainval.shape[0]
val_size = int(trainval_size * 0.1) # FIXME
val_set = dataset.DataSet(X_trainval[:val_size], y_trainval[:val_size])
train_set = dataset.DataSet(X_trainval[val_size:], y_trainval[val_size:])

In [7]:
hp_d = dict()

In [8]:
hp_d['batch_size'] = 2
hp_d['num_epochs'] = 50
hp_d['init_learning_rate'] = 1e-4
hp_d['momentum'] = 0.9
hp_d['learning_rate_patience'] = 10
hp_d['learning_rate_decay'] = 0.1
hp_d['eps'] = 1e-8
hp_d['score_threshold'] = 1e-4
hp_d['nms_flag'] = True

In [9]:
graph = tf.get_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [10]:
with tf.device('/device:GPU:1'):
  model = ConvNet([IM_SIZE[0], IM_SIZE[1], 3], NUM_CLASSES, anchors, grid_size=(IM_SIZE[0]//32, IM_SIZE[1]//32))

  evaluator = Evaluator()
  optimizer = Optimizer(model, train_set, evaluator, val_set=val_set, **hp_d)


layer1.shape [None, 208, 208, 32]
layer2.shape [None, 104, 104, 64]
layer3.shape [None, 104, 104, 128]
layer4.shape [None, 104, 104, 64]
layer5.shape [None, 52, 52, 128]
layer6.shape [None, 52, 52, 256]
<tf.Variable 'layer7/biases:0' shape=(128,) dtype=float32_ref>
layer7.shape [None, 52, 52, 128]
layer8.shape [None, 26, 26, 256]
layer9.shape [None, 26, 26, 512]
layer10.shape [None, 26, 26, 256]
layer11.shape [None, 26, 26, 512]
layer12.shape [None, 26, 26, 256]
layer13.shape [None, 13, 13, 512]
layer14.shape [None, 13, 13, 1024]
layer15.shape [None, 13, 13, 512]
layer16.shape [None, 13, 13, 1024]
layer17.shape [None, 13, 13, 512]
layer18.shape [None, 13, 13, 1024]
layer19.shape [None, 13, 13, 1024]
layer20.shape [None, 13, 13, 1024]
layer21.shape [None, 13, 13, 1280]
layer22.shape [None, 13, 13, 1024]
<tf.Variable 'biases:0' shape=(425,) dtype=float32_ref>
pred.shape [None, 13, 13, 5, 85]


In [ ]:
sess = tf.Session(graph=graph, config=config)
train_results = optimizer.train(sess, details=True, verbose=True, **hp_d)

Running training loop...
Number of training iterations: 112500
0.4162180423736572
0.01073455810546875
0.009982109069824219
0.009948015213012695
0.010084152221679688
0.010018110275268555
0.010113000869750977
0.010809898376464844
0.010663986206054688
0.010711908340454102
0.010903358459472656
0.010637760162353516
0.010716676712036133
0.010825872421264648
0.010541439056396484
0.010781526565551758
0.010663747787475586
0.010397672653198242
0.010268926620483398
0.010283946990966797
0.010638952255249023
0.010850906372070312
0.010592222213745117
0.010530948638916016
0.01865839958190918
0.010286331176757812
0.01041865348815918
0.010071039199829102
0.010032415390014648
0.010199785232543945
0.010211706161499023
0.010134696960449219
0.010149955749511719
0.010053157806396484
0.010126113891601562
0.010342121124267578
0.010647296905517578
0.010875463485717773
0.010912418365478516
0.010411739349365234
0.01063847541809082
0.010611534118652344
0.011101245880126953
0.011044502258300781
0.01050448417663574

0.011101961135864258
0.01116490364074707
0.011613607406616211
0.010904788970947266
0.010763883590698242
0.011085987091064453
0.010737419128417969
0.0110015869140625
0.01089024543762207
0.010666131973266602
0.011063098907470703
0.010840415954589844
0.010729312896728516
0.010985851287841797
0.010643720626831055
0.010637998580932617
0.010934591293334961
0.010431051254272461
0.01058816909790039
0.010444164276123047
0.010777473449707031
0.010339975357055664
0.010531425476074219
0.0105133056640625
0.010997533798217773
0.011713743209838867
0.011180400848388672
0.01114511489868164
0.010534524917602539
0.010548830032348633
0.010736942291259766
0.010903120040893555
0.010448694229125977
0.010833740234375
0.010793447494506836
0.01070713996887207
0.010516881942749023
0.011335372924804688
0.011135101318359375
0.010726213455200195
0.01061105728149414
0.010798931121826172
0.010400533676147461
0.011528730392456055
0.011292457580566406
0.01179194450378418
0.011461257934570312
0.011128664016723633
0.0106

0.011785507202148438
0.01220393180847168
0.011739015579223633
0.011733293533325195
0.01102447509765625
0.011284589767456055
0.012280941009521484
0.011644363403320312
0.011707544326782227
0.011582136154174805
0.01168370246887207
0.011410236358642578
0.011574268341064453
0.010987997055053711
0.011632919311523438
0.01140904426574707
0.011209964752197266
0.011595964431762695
0.012088298797607422
0.011588096618652344
0.011595010757446289
0.011815309524536133
0.012048721313476562
0.01204228401184082
0.011882543563842773
0.011504411697387695
0.011512517929077148
0.011743783950805664
0.011792898178100586
0.011436700820922852
0.011790037155151367
0.01184701919555664
0.011035442352294922
0.01153707504272461
0.011782646179199219
0.012470006942749023
0.011733293533325195
0.011653900146484375
0.011106729507446289
0.01151132583618164
0.011805295944213867
0.011521577835083008
0.012005329132080078
0.01226496696472168
0.012022733688354492
0.011953592300415039
0.012137174606323242
0.011785507202148438
0

0.011728525161743164
0.012215137481689453
0.0124969482421875
0.012097358703613281
0.01187586784362793
0.012352705001831055
0.012482404708862305
0.013041973114013672
0.012237071990966797
0.01215982437133789
0.011982202529907227
0.011772871017456055
0.011821508407592773
0.01124262809753418
0.01187753677368164
0.01197052001953125
0.011579513549804688
0.011746644973754883
0.012093544006347656
0.012060165405273438
0.011603355407714844
0.012264251708984375
0.011820316314697266
0.011785268783569336
0.011965274810791016
0.012151479721069336
0.01170802116394043
0.012542009353637695
0.012562990188598633
0.012011528015136719
0.012187480926513672
0.012174844741821289
0.01257634162902832
0.012175321578979492
0.01191401481628418
0.012427091598510742
0.01221013069152832
0.012161493301391602
0.011688947677612305
0.012167692184448242
0.012210369110107422
0.012083292007446289
0.01187443733215332
0.011601686477661133
0.012122631072998047
0.013114452362060547
0.012235164642333984
0.011810779571533203
0.01

0.011434316635131836
0.01148676872253418
0.011876821517944336
0.011452913284301758
0.011747121810913086
0.01155543327331543
0.012042522430419922
0.011898279190063477
0.012475728988647461
0.012301921844482422
0.012125492095947266
0.012124061584472656
0.011916160583496094
0.011935949325561523
0.011790037155151367
0.011710405349731445
0.01160883903503418
0.011940479278564453
0.012592792510986328
0.011901378631591797
0.011501073837280273
0.011313438415527344
0.012586355209350586
0.012580156326293945
0.01179361343383789
0.012284994125366211
0.01191568374633789
0.012041568756103516
0.01200723648071289
0.01160120964050293
0.011336088180541992
0.011306524276733398
0.011098146438598633
0.011570215225219727
0.011891841888427734
0.012547969818115234
0.012148141860961914
0.011927127838134766
0.011363744735717773
0.01224970817565918
0.012243509292602539
0.01159358024597168
0.01214289665222168
0.012070894241333008
0.011918783187866211
0.01220703125
0.012651681900024414
0.012452363967895508
0.0117132

...

0.011060476303100586
0.011676788330078125
0.012584686279296875
0.011095046997070312
0.012484073638916016
0.010748863220214844
0.011865854263305664
0.011084556579589844
0.011834144592285156
0.010875940322875977
0.01195669174194336
0.011008977890014648
0.01182866096496582
0.011176586151123047
0.012044906616210938
0.011103391647338867
0.011770963668823242
0.011027336120605469
0.012080907821655273
0.01124262809753418
0.012267589569091797
0.010889768600463867
0.011940956115722656
0.010988712310791016
0.011763572692871094
0.011223554611206055
0.010871410369873047
0.010648012161254883
0.01124119758605957
0.010729074478149414
0.011954069137573242
0.010512113571166992
0.011553049087524414
0.010597944259643555
0.011500120162963867
0.010471343994140625
0.01166677474975586
0.010331153869628906
0.011577367782592773
0.01040196418762207
0.011612653732299805
0.010280847549438477
0.010744571685791016
0.010292530059814453
0.010803461074829102
0.010466814041137695
0.01116037368774414
0.010387659072875977

0.01070857048034668
0.012020111083984375
0.011002540588378906
0.011986255645751953
0.01107025146484375
0.011698246002197266
0.01071929931640625
0.011644124984741211
0.01077413558959961
0.011481523513793945
0.011174678802490234
0.012369155883789062
0.011499166488647461
0.012172698974609375
0.011363029479980469
0.01239633560180664
0.011371374130249023
0.012227773666381836
0.01137685775756836
0.01226806640625
0.011291980743408203
0.012546539306640625
0.011046171188354492
0.012296915054321289
0.01158452033996582
0.01211857795715332
0.011374950408935547
0.012511014938354492
0.011191606521606445
0.012212514877319336
0.011254549026489258
0.012008905410766602
0.011260986328125
0.01209259033203125
0.011157989501953125
0.011995792388916016
0.011129617691040039
0.012420415878295898
0.011070489883422852
0.012125968933105469
0.011420011520385742
0.011781692504882812
0.010937929153442383
0.012033939361572266
0.010726690292358398
0.011827468872070312
0.011428356170654297
0.011992454528808594
0.010836

0.010838747024536133
0.011099576950073242
0.011029958724975586
0.011736869812011719
0.01166081428527832
0.011745452880859375
0.011877298355102539
0.011924505233764648
0.011752843856811523
0.012173652648925781
0.012299776077270508
0.011715173721313477
0.011751651763916016
0.012846708297729492
0.011778831481933594
0.011954069137573242
0.01216578483581543
0.011669397354125977
0.012618064880371094
0.01240682601928711
0.012094259262084961
0.011926651000976562
0.012188911437988281
0.011658668518066406
0.011939525604248047
0.012464046478271484
0.01222538948059082
0.011928081512451172
0.012050390243530273
0.011708736419677734
0.011587142944335938
0.011556625366210938
0.011249542236328125
0.010849237442016602
0.011048316955566406
0.011403083801269531
0.010892391204833984
0.010834217071533203
0.010859489440917969
0.01073765754699707
0.010852336883544922
0.010823726654052734
0.01077723503112793
0.011262893676757812
0.011019468307495117
0.010923385620117188
0.010371208190917969
0.01046943664550781

0.011877059936523438
0.012356281280517578
0.012395381927490234
0.011401176452636719
0.011408090591430664
0.011631250381469727
0.011751890182495117
0.011641502380371094
0.011928081512451172
0.012061595916748047
0.012248754501342773
0.012059211730957031
0.012279510498046875
0.010972976684570312
0.011670589447021484
0.01206827163696289
0.011801719665527344
0.011609554290771484
0.01221776008605957
0.012209653854370117
0.012076139450073242
0.011915206909179688
0.012538671493530273
0.012048959732055664
0.012047290802001953
0.0119476318359375
0.011677980422973633
0.01174163818359375
0.011572122573852539
0.011546611785888672
0.01169276237487793
0.01148223876953125
0.01223134994506836
0.011664390563964844
0.011669397354125977
0.012186527252197266
0.01171255111694336
0.011685371398925781
0.011644840240478516
0.011591911315917969
0.011555194854736328
0.011563777923583984
0.011610984802246094
0.01118922233581543
0.011576414108276367
0.011919975280761719
0.012129545211791992
0.012001276016235352
0.

0.010440826416015625
0.01120758056640625
0.010139226913452148
0.010908365249633789
0.010332822799682617
0.010954141616821289
0.010513067245483398
0.010953664779663086
0.011055707931518555
0.01191854476928711
0.01079249382019043
0.011654376983642578
0.011374473571777344
0.01185154914855957
0.011199235916137695
0.01178741455078125
0.011018514633178711
0.011849403381347656
0.011180877685546875
0.011965513229370117
0.010736227035522461
0.011345148086547852
0.010976791381835938
0.01225733757019043
0.010889768600463867
0.011110067367553711
0.010998964309692383
0.011242151260375977
0.01065373420715332
0.010898113250732422
0.010572195053100586
0.011854410171508789
0.01020359992980957
0.010779142379760742
0.010081291198730469
0.011483192443847656
0.01042318344116211
0.011895179748535156
0.010833024978637695
0.01116633415222168
0.01091766357421875
0.011676549911499023
0.010791301727294922
0.011017322540283203
0.010042905807495117
0.010953187942504883
0.01018524169921875
0.010845661163330078
0.01

x = "Hello World!"
zfill_amount = 20
filled = x.zfill(zfill_amount)
print(filled)
restored = filled[zfill_amount-len(x):]
print(restored)

asd="00000001234"
print(asd)

print(type(asd))

qwe=asd.lstrip('0')

print(qwe)

print(type(X_trainval))
print(len(X_trainval))
print(X_trainval.shape)

print(type(y_trainval))
print(len(y_trainval))
print(y_trainval.shape)

print(type())
print(len())